In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from nltk import tokenize
from nltk.corpus import stopwords
import re
import contractions

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/psicurello/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
data = pd.read_csv('cleaned_data.csv')

In [4]:
data

,Unnamed: 0,Unnamed: 0.1,Summary,Text,Clean Text,Clean Summary,Lemmatized Summary,Lemmatized Text
0,0,0,"Keep related supplies in the same area.,Make a...","If you're a photographer, keep all the necess...",photographer keep necessary lens cords batteri...,keep related supplies area make effort clean d...,related supply area effort clean dedicated wor...,photographer necessary lens cord battery quadr...
1,1,1,Create a sketch in the NeoPopRealist manner of...,See the image for how this drawing develops s...,see image drawing develops step step however i...,create sketch neopoprealist manner future mura...,create sketch neopoprealist manner future mura...,image drawing develop step step important deta...
2,2,2,"Get a bachelor’s degree.,Enroll in a studio-ba...",It is possible to become a VFX artist without...,possible become vfx artist without college deg...,get bachelor degree enroll studio based progra...,bachelor degree enroll studio base program tra...,possible vfx artist college degree path easy v...
3,3,3,Start with some experience or interest in art....,The best art investors do their research on t...,the best art investors research pieces art buy...,start experience interest art understand diffe...,start experience interest art understand diffe...,good art investor research piece art buy educa...
4,4,4,"Keep your reference materials, sketches, artic...","As you start planning for a project or work, ...",start planning project work likely gathering s...,keep reference materials sketches articles pho...,reference material sketch article photo etc ea...,start plan project work likely gather scrap in...
...,...,...,...,...,...,...,...,...
214288,215360,215360,"Consider changing the spelling of your name.,A...","If you have a name that you like, you might f...",name like might fiddle spelling see alternate ...,consider changing spelling name avoid symbols ...,consider change spelling avoid symbol add exot...,like fiddle spelling alternate letter interest...
214289,215361,215361,"Try out your name.,Don’t legally change your n...",Your name might sound great to you when you s...,your name might sound great say loud bedroom f...,try name legally change name register stage na...,try legally change register stage trade guild ...,sound great loud bedroom find sound announce t...
214290,215362,215362,"Understand the process of relief printing.,Exa...",Relief printing is the oldest and most tradit...,relief printing oldest traditional printing te...,understand process relief printing examine rim...,understand process relief printing examine rim...,relief print oldest traditional print technolo...
214291,215363,215363,"Understand the process of intaglio printing.,L...","Intaglio is Italian for ""incis­ing,"" and corr...",intaglio italian incis ing correspondingly rev...,understand process intaglio printing look plat...,understand process intaglio print look plate m...,intaglio italian incis ing correspondingly rev...


In [5]:
abridged = data[:10]

### Data Preprocessing

In [8]:
""" Here we set text to lower case, remove plurals, 
    expand contractions, remove punctuation, remove stopwords, and remove short words 
    (could also remove parentheticals)"""

stop = set(stopwords.words('english'))
def clean_text(text):
    ret = text.lower()
    ret = contractions.fix(text)
    ret = re.sub(r'\([^)]*\)', '', ret)
    ret = re.sub('"','', ret)
    ret = re.sub(r"'s\b","", ret)
    ret = re.sub("[^a-zA-Z]", " ", ret) 
    
    #Remove any words shorter than 2 letters
    tokens = [w for w in ret.split() if not w in stop]
    long_words=[]
    for i in tokens:
        if len(i)>=3:                 
            long_words.append(i)   
    return (" ".join(long_words)).strip()

In [101]:
test = data['Text'][0]
text = test.replace('\\', '').replace('/', '').replace('.,', '.').replace('.;,', '.')

In [84]:
from nltk.tokenize import sent_tokenize
articles = []
#for s in abridged['Text']:
#    articles.append(sent_tokenize(s))  

sentence = sent_tokenize(text)
clean = []
for sen in sentence:
    clean.append(clean_text(sen))
"""clean_articles = []
for a in articles:
    temp = []
    for sen in a:
        temp.append(clean_text(sen))
    clean_articles.append(temp)"""

'clean_articles = []\nfor a in articles:\n    temp = []\n    for sen in a:\n        temp.append(clean_text(sen))\n    clean_articles.append(temp)'

In [112]:
#Word2Vec
from gensim.models import Word2Vec
from nltk.corpus import stopwords

words = []
all_words = [i.split() for i in clean]
model = Word2Vec(all_words, min_count=1, vector_size=300)

sent_vector=[]
for i in clean:
    plus=0
    for j in i.split():
        plus+= model.wv[j]
    plus = plus/len(i.split())
    sent_vector.append(plus)

"""sent_vector=[]
for i in clean_articles[1]:
    plus=0
    for j in i.split():
        plus+= model.wv[j]
    plus = plus/len(i.split())
    sent_vector.append(plus)"""

'sent_vector=[]\nfor i in clean_articles[1]:\n    plus=0\n    for j in i.split():\n        plus+= model.wv[j]\n    plus = plus/len(i.split())\n    sent_vector.append(plus)'

### KMeans

In [96]:
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial import distance

n_clusters = 5
kmeans = KMeans(n_clusters, init = 'k-means++', random_state = 42)
y_kmeans = kmeans.fit_predict(sent_vector)

my_list=[]
for i in range(n_clusters):
    my_dict={}
    
    for j in range(len(y_kmeans)):
        
        if y_kmeans[j]==i:
            my_dict[j] = distance.euclidean(kmeans.cluster_centers_[i],sent_vector[j])
    min_distance = min(my_dict.values())
    my_list.append(min(my_dict, key=my_dict.get))
 
                            
for i in sorted(my_list):
    print(sentence[i])

clean each night.
Endeavor to leave the workspace in a way that you can sit down the next day and start working immediately, without having to do any work or tidying.Even if the rest of your studio is a bit disorganized, an organized workspace will help you get down to business every time you want to make art.
Once a month, do a purge of your studio.
Artists are constantly making new things, experimenting, and making a mess.
Toss it.


### TextRank

In [110]:
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

#Generate Similarity Matrix
mat = np.zeros([len(sentence), len(sentence)])
for i in tqdm(range(len(sentence))):
    for j in range(len(sentence)):
        if i != j:
            mat[i][j] = cosine_similarity(sent_vector[i].reshape(1,-1), sent_vector[j].reshape(1, -1))[0,0]

100%|██████████| 29/29 [00:00<00:00, 186.29it/s]

(29, 29)


In [117]:
import networkx as net

#Implement TextRank using PageRank
graph = net.from_numpy_array(mat)
scores = net.pagerank_numpy(graph)

<ipython-input-117-3e6bd4173755>:5: DeprecationWarning: networkx.pagerank_numpy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  scores = net.pagerank_numpy(graph)


In [123]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentence)), reverse=True)
#for i in range(10):
#    print(ranked_sentences[i])

result = ""
for i in range(8):
    result += " " + ranked_sentences[i][1]
print(result)

 clean each night. Cheap and easy, this is also a good way to handle papers and ideas you touch regularly or need to pin up and down for inspiration. You can even use it for smaller areas. Paints should be kept with brushes, cleaner, and canvas, print supplies should be by the ink, etc. Endeavor to leave the workspace in a way that you can sit down the next day and start working immediately, without having to do any work or tidying.Even if the rest of your studio is a bit disorganized, an organized workspace will help you get down to business every time you want to make art. Shelving is an artist's best friend and is a cheap and easy way to get more room in your studio or art space. Once a month, do a purge of your studio. Turning one wall into a chalkboard gives you a perfect space for ideas, sketches, and planning without requiring extra equipment or space.


### Huggingface

In [124]:
from transformers import pipeline
import os

In [128]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
summarizer = pipeline("summarization")
summary_text = summarizer(text, max_length=100, min_length=5, do_sample=False)[0]['summary_text']
print(summary_text)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


 A lot of artist clutter comes from a desire to keep track of supplies visually instead of tucked out of sight . By using jars, old glasses, vases, and cheap, clear plastic drawers, you can keep things in sight without leaving it strewn about haphazardly . Shelving is an artist's best friend and is a cheap and easy way to get more room .


In [133]:
!pip3 install sentencepiece

  Using cached sentencepiece-0.1.96-cp38-cp38-macosx_10_6_x86_64.whl (1.1 MB)


We see that both our unsupervised extractive models perform fairly poorly. I first used embedded the sentences using Word2Vec, and then tried both a TextRank and KMeans algorithm to find the "most important" sentences in the article. This produced sub-par results, with KMeans's producing a barely legible paragraph and TextRank performing slightly better.

Then, for benchmarking purposes, I used the default distilbart-cnn-12-6 model from huggingface's model repo and nlp pipeline to summarize the same article. The results were much better, indicating that I should proceed with abstractive summarization.